### Imports

In [1]:
import requests 
import json 
import pandas as pd
from bs4 import BeautifulSoup
from mplsoccer import Pitch, VerticalPitch
import numpy as np
from matplotlib.patches import FancyArrowPatch
from matplotlib.offsetbox import OffsetImage
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
from matplotlib.patches import ConnectionPatch
import os

import urllib.request
from PIL import Image

import ScraperFC as sfs
from io import BytesIO

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Fonts and colors 
from matplotlib import font_manager
locations = ['/Users/emilio/Documents/3- Resources/lightstats/fonts/Montserrat']
font_files = font_manager.findSystemFonts(fontpaths=locations)
for file in font_files:
    font_manager.fontManager.addfont(file)
font = 'Montserrat'

# Define the colors using hexadecimal representations
background_color = '#F7F7F7' 
title_color = '#000000'   
text_color = '#333333'  
sub_text_color = '#808080'   
accent_color1 = '#4682B4'    
accent_color2 = '#ADD8E6' 

### Scrap info

In [4]:
# Scrap sofascore
sofascore = sfs.Sofascore()


sofascore_url = 'https://www.sofascore.com/pec-zwolle-feyenoord/jjbswjb#id:11388402'
sofascore_url = sofascore_url.replace('id:', '')


# Scrap fotmob
fotmob_url = 'https://www.fotmob.com/api/matchDetails?matchId=4216652'

payload = {}
headers = {}

response = requests.request("GET", fotmob_url, headers=headers, data=payload)

data = response.json()

# print(response.text) Imprime el html

### DataFrames

In [5]:
dfShots = pd.DataFrame(response.json()['content']['shotmap']['shots'])
df_matchStats = sofascore.get_general_match_stats(sofascore_url)

In [6]:
# Df's for the heatmap
dfhome = sofascore.get_players_match_stats(sofascore_url)[0]
dfaway = sofascore.get_players_match_stats(sofascore_url)[1]

home_posDf = sofascore.get_players_average_positions(sofascore_url)[0]
homePositions = home_posDf[['jerseyNumber', 'averageX', 'averageY']]
homename = home_posDf['team'].unique()[0]

away_posDf = sofascore.get_players_average_positions(sofascore_url)[1]
awayPositions = away_posDf[['jerseyNumber', 'averageX', 'averageY']]
awayname = away_posDf['team'].unique()[0]

teamsDf = pd.concat([home_posDf, away_posDf])

KeyError: 'captain'

### Team's Stats

In [ ]:
# Extracting the 'stats' dictionary from the 'expected_goals_stats' dictionary
expected_goals_stats = data['content']['stats']['Periods']['All']['stats'][2]['stats'][1]
# Extracting the array of expected goals values from the 'expected_goals_stats' dictionary
expected_goals_array = expected_goals_stats['stats']


# Lista de todas las estadísticas
all_stats = ['xG', 'Possesion', 'Shots', 'Fouls', 'Big Chances']

# Obtener los datos para el equipo home
home_data = {
    'xG': float(expected_goals_array[0]),
    'Possesion': float(df_matchStats[df_matchStats['name'] == 'Ball possession']['homeValue'].iloc[0]),
    'Shots': int(df_matchStats[df_matchStats['name'] == 'Total shots']['home'].iloc[0]),
    'Fouls': int(df_matchStats[df_matchStats['name'] == 'Fouls']['home'].iloc[0]),
    'Big Chances': float(df_matchStats[df_matchStats['name'] == 'Big chances']['home'].iloc[0])
}

# Obtener los datos para el equipo visitante
away_data = {
    'xG': float(expected_goals_array[1]),
    'Possesion': float(df_matchStats[df_matchStats['name'] == 'Ball possession']['awayValue'].iloc[0]),
    'Shots': int(df_matchStats[df_matchStats['name'] == 'Total shots']['away'].iloc[0]),
    'Fouls': int(df_matchStats[df_matchStats['name'] == 'Fouls']['away'].iloc[0]),
    'Big Chances': float(df_matchStats[df_matchStats['name'] == 'Big chances']['away'].iloc[0])
}

In [ ]:
def get_opponent_team(df, playerName):
    # Filter rows where playerName matches
    player_df = df[df['name'] == playerName]

    # Get the player's team name
    player_team = player_df['team'].unique()[0]

    # Find the opponent team
    opponent_team = df[df['team'] != player_team]['team'].unique()[0]

    return opponent_team

### Game Data

In [ ]:
round = data['general']['leagueRoundName']
round = round.replace('Round of 16', 'Octavos de Final')
round = round.replace('Round', 'Jornada')
round = round.replace('Semi-Finals', 'Semifinal')
round = round.replace('Quarter-Finals', 'Cuartos de Final')

match_time = data['general']['matchTimeUTC']
season = data['general']['parentLeagueSeason']
score = data['header']['status']['scoreStr']
pen_flag = data['header']['status']['reason']['short']
league = sofascore.get_match_data(sofascore_url)['tournament']['uniqueTournament']['name']

if pen_flag == 'Pen':
    away_penalties = data['header']['status']['reason']['penalties'][1]
    home_penalties = data['header']['status']['reason']['penalties'][0]

### Team's Data

In [ ]:
# Check if 'general' key exists in the response
home_name = data['header']['teams'][0]['name']
home_id = data['header']['teams'][0]['id']
home_color = dfShots[dfShots['teamId'] == home_id]['teamColor'].iloc[0]
home_img_url = data['header']['teams'][0]['imageUrl']

away_name = data['header']['teams'][1]['name']
away_id = data['header']['teams'][1]['id']
away_color = dfShots[dfShots['teamId'] == away_id]['teamColor'].iloc[0]
away_img_url = data['header']['teams'][1]['imageUrl']

team_data = {}
# Function to populate team data
def populate_team_data(team_id, data, df_shots):
    team_index = 0 if team_id == data['header']['teams'][0]['id'] else 1
    team_name = data['header']['teams'][team_index]['name']
    team_color = df_shots[df_shots['teamId'] == team_id]['teamColor'].iloc[0]
    team_img_url = data['header']['teams'][team_index]['imageUrl']
    return team_name, team_color, team_img_url

# Populate team data
home_name, home_color, home_img_url = populate_team_data(home_id, data, dfShots)
away_name, away_color, away_img_url = populate_team_data(away_id, data, dfShots)

In [ ]:
""" Get if the colors are similar """

def get_rgb(color):
    """Converts a hex color to an RGB tuple."""
    color = int(color[1:], 16)
    r = color >> 16
    g = (color - (r << 16)) >> 8
    b = color - (r << 16) - (g << 8)
    return [r, g, b]

def is_similar(color1, color2):
    """Checks if two colors are similar."""
    r1, g1, b1 = get_rgb(color1)
    r2, g2, b2 = get_rgb(color2)
    return (abs(r1 - r2) + abs(g1 - g2) + abs(b1 - b2)) < 50


if is_similar(home_color, away_color):
    home_color = '#FF0000'
    away_color = '#0000ff'

### Shotmap

In [ ]:
# Filtrar los disparos del equipo home y del equipo visitante
home_shots = dfShots[dfShots['teamId'] == home_id]
away_shots = dfShots[dfShots['teamId'] == away_id]

# Filtrar los goles del equipo home y mostrarlos de manera diferente
home_goal_shots = home_shots[home_shots['eventType'] == 'Goal']
away_goal_shots = away_shots[away_shots['eventType'] == 'Goal']

# Filtrar los disparos que no son goles del equipo home y mostrarlos con transparencia
home_non_goal_shots = home_shots[home_shots['eventType'] != 'Goal']
away_non_goal_shots = away_shots[away_shots['eventType'] != 'Goal']

### POM Stats

#### Fotmob

In [ ]:
dfPom = data['content']['matchFacts']['playerOfTheMatch']

# Extract the player of the match details
pom_stats = dfPom['stats']

# POM = Player of the Match
pomFM = dfPom['name']['fullName']
pom_position = dfPom['role']
pom_id = dfPom['id']
pom_team = dfPom['teamName']
pom_img_url = f'https://images.fotmob.com/image_resources/playerimages/{pom_id}.png'

# Initialize empty lists to store the keys and values for plotting
keys = []
values = []

positions = ['Keeper', 'Defender', 'Midfielder', 'Attacker']
attackerKeys = ['Goals', 'Assist', 'Total shots','Expected goals (xG)', 'Shot accuracy', 'Successful dribbles', 'xG Non-penalty']
midfielderKeys = ['Goals', 'Assists', 'Expected assists (xA)', 'Accurate passes', 'Chances created', 'Accurate long balls', 'Passes into final third', 'Successful dribbles']
defenderKeys = ['Tackles won', 'Blocks', 'Recoveries', 'Interceptions', 'Clearances']
keeperKeys = ['Clearances', 'Recoveries', 'Defensive actions']

# Define a dictionary to map positions to keys
position_keys_mapping = {
    'Keeper': keeperKeys,
    'Defender': defenderKeys,
    'Midfielder': midfielderKeys,
    'Attacker': attackerKeys
}

if pom_position in positions:
    selected_keys = position_keys_mapping[pom_position]
    print("Selected keys:", selected_keys)  # Debugging statement

    # Iterate over the stats dictionary to extract the specified keys and values
    for stat_group in pom_stats:
        # Check if the title matches the desired titles
        if stat_group['title'] in ['Top stats', 'Attack', 'Defense']:
            # Extract the 'stats' dictionary
            stats_dict = stat_group['stats']
            
            # Iterate over the keys and store the values for specified keys
            for key in selected_keys:
                if key in stats_dict:
                    keys.append(key)
                    # Convert value to a numeric type
                    value = stats_dict[key]['stat']['value']
                    print("Value for", key, ":", value)  # Debugging statement
                    if isinstance(value, str) and '/' in value:  # Handling cases like '28/40 (70%)'
                        value = float(value.split('/')[0])  # Extracting the numerator as a float
                    else:
                        value = float(value)  # Convert to float if possible

                    values.append(value)

Selected keys: ['Goals', 'Assist', 'Total shots', 'Expected goals (xG)', 'Shot accuracy', 'Successful dribbles', 'xG Non-penalty']
Value for Goals : 2
Value for Total shots : 5
Value for Expected goals (xG) : 1.14
Value for Shot accuracy : 3
Value for Successful dribbles : 2
Value for xG Non-penalty : 0.35


In [8]:
# Print the keys of Stats 
def pom_statsKeys(title):
    for stat_group in pom_stats:
        if stat_group['title'] == title:
            print(f"Stats in {title}: ")
            for key, value in stat_group['stats'].items():
                print(f'- {key}')

#### SofaScore

In [9]:
max_rating_home = dfhome['rating'].idxmax()
pomhome = dfhome.loc[max_rating_home, 'name']
pom_homeRating = dfhome['rating'].max()

max_rating_away = dfaway['rating'].idxmax()
pomAway = dfaway.loc[max_rating_away, 'name']
pom_awayRating = dfaway['rating'].max()

if pom_homeRating > pom_awayRating:
    pomSS = pomhome
    pom_rating = pom_homeRating
else:
    pomSS = pomAway
    pom_rating = pom_awayRating

opponent_team = get_opponent_team(teamsDf, pomSS)
pom_heatmap = sofascore.get_player_heatmap(sofascore_url, pomSS)

NameError: name 'dfaway' is not defined

### Momentum

In [ ]:
match_momentum = sofascore.match_momentum(sofascore_url)
momentum = sofascore.match_momentum(sofascore_url)

match_momentum_home = match_momentum[match_momentum['value'] > 0]
match_momentum_visit = match_momentum[match_momentum['value'] < 0]

### Player Shots

#### Data

In [ ]:
""" Print the names of the players that shoot at least one time"""
away_players_shooting = away_shots['playerName'].unique()
home_players_shooting = home_shots['playerName'].unique()

print(home_players_shooting)
print(away_players_shooting)

['David Hancko' 'Luka Ivanusec' 'Igor Paixao' 'Ayase Ueda' 'Calvin Stengs'
 'Quinten Timber' 'Santiago Gimenez' 'Gernot Trauner'
 'Alireza Jahanbakhsh' 'Thomas van den Belt' 'Lutsharel Geertruida'
 'Ondrej Lingr']
['Silvester van der Water' 'Damian van der Haar' 'Odysseus Velanas']


In [ ]:
#player_name = 'Marcus Rashford'
dfPom = data['content']['matchFacts']['playerOfTheMatch']
pom_positionFM = dfPom['role']

# Could change pomFM to player_name
player_shots = dfShots[dfShots['playerName'] == pomFM]
player_goal_shots = player_shots[player_shots['eventType'] == 'Goal']
player_non_goal_shots = player_shots[player_shots['eventType'] != 'Goal']

In [ ]:
playerId = dfPom['id']
player_img_url = f'https://images.fotmob.com/image_resources/playerimages/{playerId}.png'

# Retrieve team data
player_team_id = dfPom['teamId']

if player_team_id == home_id:
    against_id = away_id
    against_name, against_color, against_img_url = away_name, away_color, away_img_url
elif player_team_id == away_id:
    against_id = home_id
    against_name, against_color, against_img_url = home_name, home_color, home_img_url

#### Plot shots

In [ ]:
""" Shots of the Selected Player during a Match """
def playerShots(season): 
    pitch = VerticalPitch(pitch_type='custom', pitch_length=105, pitch_width=68, half=True, pitch_color=background_color)
    fig, ax = pitch.draw()

    fig.patch.set_facecolor(background_color)

    if not player_goal_shots.empty:
        pitch.scatter(player_goal_shots.x, player_goal_shots.y, ax=ax, marker='football', s=200)
    if not player_non_goal_shots.empty:
        pitch.scatter(player_non_goal_shots.x, player_non_goal_shots.y, ax=ax, alpha=0.6, color='black')

    # Fetch and plot the home team image (logo)
    response = requests.get(player_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax_logo = fig.add_axes([0, 0.95, 0.1, 0.1])  # [x, y, Width, Height]
    ax_logo.imshow(img_data)
    ax_logo.axis(False)

    fig.text(0.1, 1, f'Tiros {pomFM} vs {against_name}', fontsize=14, weight='bold', fontfamily=font, color=title_color)
    fig.text(0.1, 0.95, f'{home_name} vs {away_name} | {round}', fontsize=12, fontfamily=font, color=sub_text_color)

    fig.text(.25, 0.1, "@3zavalam", fontsize=10, ha='right', fontfamily=font, color=text_color)
    fig.text(.75, 0.1, " Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "match", league, season, round, f'{home_name}_{away_name}')
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_player = os.path.join(directory, f"tiros_{pomFM}.png")
    plt.savefig(file_path_player, bbox_inches='tight') 
    
    plt.close()

    return file_path_player

### Team Shots

In [ ]:
# Contar el número de own goals del equipo home y visitante
away_own_goals_count = away_shots[away_shots['isOwnGoal'] == True]['isOwnGoal'].count()
home_own_goals_count = home_shots[home_shots['isOwnGoal'] == True]['isOwnGoal'].count()

# Restar el número de own goals del equipo contrario
away_goals = (away_shots[away_shots['eventType'] == 'Goal']['eventType'].count() - away_own_goals_count) + home_own_goals_count
home_goals = (home_shots[home_shots['eventType'] == 'Goal']['eventType'].count() - home_own_goals_count) + away_own_goals_count

In [ ]:
""" Shots of the home team"""
def homeShots(season):
    pitch = VerticalPitch(pitch_type='custom', pitch_length=105, pitch_width=68, half=True, pitch_color=background_color)
    fig, ax = pitch.draw()

    fig.patch.set_facecolor(background_color)

    if not home_goal_shots.empty:
        pitch.scatter(home_goal_shots.x, home_goal_shots.y, ax=ax, marker='football', s=200)
    if not home_non_goal_shots.empty:
        pitch.scatter(home_non_goal_shots.x, home_non_goal_shots.y, ax=ax, alpha=0.6, label=home_name, color=home_color)

    # Fetch and plot the home team image (logo)
    response = requests.get(home_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax_logo = fig.add_axes([0, 0.95, 0.1, 0.1])  # [x, y, Width, Height]
    ax_logo.imshow(img_data)
    ax_logo.axis(False)

    fig.text(0.1, 1, f'Tiros {home_name} vs {away_name}', fontsize=16, weight='bold', fontfamily=font, color=text_color)
    fig.text(0.1, 0.95, f'{home_name} vs {away_name} | {round}', fontsize=13, fontfamily=font, color=sub_text_color)

    fig.text(.25, 0.1, "@3zavalam", fontsize=10, ha='right', fontfamily=font, color=text_color)
    fig.text(.75, 0.1, " Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

    # Data about the shots
    fig.text(.92, .6, f'{home_data['xG']}', fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .6, 'xG', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .5, f'{home_goals}', fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .5, 'Goles', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .4, f'{home_shots['eventType'].count()}', fontsize=14, fontfamily=font,  weight='bold', color=text_color)
    fig.text(1, .4, 'Tiros', fontfamily=font, color=text_color, fontsize=14)
    
    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "match", league, season, round, f'{home_name}_{away_name}')
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_homeShots = os.path.join(directory, f"tiros_{home_name}.png")
    plt.savefig(file_path_homeShots, bbox_inches='tight') 
    plt.close()

    return file_path_homeShots

In [ ]:
""" Shots of the Away team"""
def awayShots(season):
    pitch = VerticalPitch(pitch_type='custom', pitch_length=105, pitch_width=68, half=True, pitch_color=background_color)
    fig, ax = pitch.draw()

    fig.patch.set_facecolor(background_color)

    if not away_goal_shots.empty:
        pitch.scatter(away_goal_shots.x, away_goal_shots.y, ax=ax, marker='football', s=200)
    if not away_non_goal_shots.empty:
        pitch.scatter(away_non_goal_shots.x, away_non_goal_shots.y, ax=ax, alpha=0.6, label=away_name, color=away_color)

    # Fetch and plot the home team image (logo)
    response = requests.get(away_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax_logo = fig.add_axes([0, 0.95, 0.1, 0.1])  # [x, y, Width, Height]
    ax_logo.imshow(img_data)
    ax_logo.axis(False)

    fig.text(0.1, 1, f'Tiros {away_name} vs {home_name}', fontsize=16, weight='bold', fontfamily=font, color=text_color)
    fig.text(0.1, 0.95, f'{home_name} vs {away_name} | {round}', fontsize=13, fontfamily=font, color=sub_text_color)

    fig.text(.25, 0.1, "@3zavalam", fontsize=10, ha='right', fontfamily=font, color=text_color)
    fig.text(.75, 0.1, " Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

    # Data about the shots
    fig.text(.92, .6, f"{away_data['xG']}", fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .6, 'xG', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .5, f'{away_goals}', fontsize=14, fontfamily=font, weight='bold', color=text_color)
    fig.text(1, .5, 'Goles', fontfamily=font, color=text_color, fontsize=14)

    fig.text(.92, .4, f'{away_shots['eventType'].count()}', fontsize=14, fontfamily=font,  weight='bold', color=text_color)
    fig.text(1, .4, 'Tiros', fontfamily=font, color=text_color, fontsize=14)


    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "match", league, season, round, f'{home_name}_{away_name}')
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_awayShots = os.path.join(directory, f"tiros_{away_name}.png")
    plt.savefig(file_path_awayShots, bbox_inches='tight') 
    plt.close()
    
    return file_path_awayShots

### Dashboard

In [ ]:
def dashboard(season):
    fig = plt.figure(figsize=(16, 9), constrained_layout=True)
    gs = fig.add_gridspec(nrows=6, ncols=9)
    fig.set_facecolor(background_color)

    # Plotting Game Data
    ax1 = fig.add_subplot(gs[0, 1:7])
    if pen_flag == 'Pen':
        ax1.text(0.5, 0.70, f'{home_name} ({home_penalties}) {score} ({away_penalties}) {away_name}', weight='bold', fontsize=25, ha='center', fontfamily=font, color=title_color)
    else:
        ax1.text(0.5, 0.70, f'{home_name} {score} {away_name}', weight='bold', fontsize=30, ha='center', fontfamily=font, color=title_color)
    ax1.text(0.5, 0.25, f'{league}: {round}', fontsize=15, ha='center', fontfamily=font, color=sub_text_color)
    ax1.axis(False)

    # Fetch and plot the home team image
    ax10 = fig.add_subplot(gs[0, :1])
    response = requests.get(home_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax10.imshow(img_data)
    ax10.axis(False)

    # Fetch and plot the away team image
    ax11 = fig.add_subplot(gs[0, 7:8])
    response = requests.get(away_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax11.imshow(img_data)
    ax11.axis(False)

    # Plotting Game Stats
    ax2 = fig.add_subplot(gs[1:3, :3])  
    # Crear la figura y los subgráficos dentro de ax2
    axs = [ax2]  # Utilizar una lista para axs para que coincida con la estructura existente
    # Bucle sobre todas las estadísticas
    for i, stat in enumerate(all_stats):
        # Calcular la suma total de la estadística
        total_stat = home_data[stat] + away_data[stat]
        
        # Calcular los porcentajes de contribución del equipo home y visitante
        porcentaje_home = home_data[stat] / total_stat
        porcentaje_visitante = away_data[stat] / total_stat
        
        # Crear la barra horizontal con dos colores
        ax2.barh(i, porcentaje_home, color=home_color, height=0.5)
        ax2.barh(i, porcentaje_visitante, color=away_color, left=porcentaje_home, height=0.5)
        
        # Añadir el valor numérico para el equipo home y visitante
        ax2.text(0.1, i, f'{home_data[stat]}', fontsize=14, ha='center', va='center', color='white', fontfamily=font)
        ax2.text(0.9, i, f'{away_data[stat]}', fontsize=14, ha='center', va='center', color='white', fontfamily=font)
        ax2.text(0.5, i, stat, fontsize=14, ha='center', va='center', color='white', fontfamily=font)
    ax2.axis(False)

    ax3 = fig.add_subplot(gs[1:3, 3:8])

    # Plot the home and away match momentum
    ax3.bar(match_momentum_home.minute, match_momentum_home.value, color=home_color)
    ax3.bar(match_momentum_visit.minute, match_momentum_visit.value, color=away_color)

    # Add text annotations for minutes 1, 46, 90, and 120 if they exist
    lowest_value = min(match_momentum_home.value.min(), match_momentum_visit.value.min())
    ax3.text(1, lowest_value - 5, '0\'', ha='right', va='center', weight='bold', fontfamily=font, color=text_color)
    ax3.text(46, lowest_value - 5, 'HT', ha='center', va='center', weight='bold', fontfamily=font, color=text_color)
    ax3.text(90, lowest_value - 5, "90'", ha='center', va='center', weight='bold', fontfamily=font, color=text_color)

    # Check if the 120th minute exists and plot it if it does
    if 120 in match_momentum_home.minute.values or 120 in match_momentum_visit.minute.values:
        ax3.text(120, lowest_value - 5, "120'", ha='center', va='center', weight='bold', fontfamily=font, color=text_color)

    # Set the title and hide axis
    ax3.set_title('Match Momentum', weight='bold')
    ax3.axis(False)


    ax4 = fig.add_subplot(gs[3:6, :3])
    y_start = 0.6
    # Plot the name and position
    ax4.text(0.3, 0.8, f'{pomFM}', weight='bold', fontsize=20, fontfamily=font, color=text_color)
    ax4.text(0.3, 0.75, f'{pom_position}, {pom_team}', fontsize=10, fontfamily=font, color=text_color)
    # Iterate over keys and values to plot them horizontally
    for i, (key, value) in enumerate(zip(keys, values)):
        # Calculate x-coordinate for the current key-value pair
        x_key = 0
        x_value = 0.6
        # Calculate y-coordinate for the current key-value pair
        y = y_start - i * 0.1
        # Make the key bold and plot it
        ax4.text(x_key, y, f'{key}:', fontsize=12, fontfamily=font, color=text_color)
        # Plot the value
        ax4.text(x_value, y, f'{value}', fontsize=12, weight= 'bold', fontfamily=font, color=text_color)
    ax4.axis(False)

    ax5 = fig.add_subplot(gs[3:4, :1])
    response = requests.get(pom_img_url)
    img_data = Image.open(BytesIO(response.content))
    ax5.imshow(img_data)
    ax5.axis(False)

    ax6 = fig.add_subplot(gs[3:, 3:8])
    ax6.text(26.25, 5,f'{home_name} shots', color=home_color, ha='center', fontsize=12, fontfamily=font)
    ax6.text(78.75, 5,f'{away_name} shots', color=away_color, ha='center', fontsize=12, fontfamily=font)
    pitch = Pitch(pitch_type='custom', pitch_length=105, pitch_width=68)
    pitch.draw(ax=ax6)
    if not home_goal_shots.empty:
        pitch.scatter(105-home_goal_shots.x, 68-home_goal_shots.y, ax=ax6, marker='football', s=200)
    if not away_goal_shots.empty:
        pitch.scatter(away_goal_shots.x, away_goal_shots.y, ax=ax6, marker='football', s=200)
    if not home_non_goal_shots.empty:
        pitch.scatter(105-home_non_goal_shots.x, 68-home_non_goal_shots.y, ax=ax6, alpha=0.6, label=home_name, color=home_color)
    if not away_non_goal_shots.empty:
        pitch.scatter(away_non_goal_shots.x, away_non_goal_shots.y, ax=ax6, alpha=0.6, label=away_name, color=away_color)
    #ax6.legend(loc='lower left')
    ax6.axis(False)

    fig.text(.63, 0.01, "Made by: @3zavalam     |    Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "match", league, season, round, f'{home_name}_{away_name}')
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_game = os.path.join(directory, f"dash_{home_name}_{away_name}.png")
    plt.savefig(file_path_game, bbox_inches='tight')

    plt.close()
    return file_path_game

### Heatmap

In [ ]:
def heatmap(season):
    fig, ax = plt.subplots(figsize=(16,9))
    fig.set_facecolor(color=background_color)

    cmap = 'hot'
    pitch = Pitch(pitch_type='opta', pitch_color=background_color, line_color=sub_text_color)
    pitch.draw(ax=ax)
    pitch.kdeplot(pom_heatmap.x, pom_heatmap.y, ax=ax,
                levels=100,
                shade=True,
                zorder=-1,
                shade_lowest=True,
                cmap='OrRd'
    )
    plt.gca()

    ax.set_title(f'{pomSS} heatmap vs {opponent_team}', fontsize=20, fontfamily=font, color=title_color, weight='bold')

    arrow = FancyArrowPatch((40, -2), (60, -2), mutation_scale=20, color=sub_text_color, arrowstyle='-|>')
    ax.add_patch(arrow)
    ax.text(50, -5, 'Ataque', fontsize=12, ha='center', fontfamily=font, color=sub_text_color)
    ax.text(100, -3, "@3zavalam | Data: SofaScore y FotMob", fontsize=10, ha='right', fontfamily=font, color=text_color)

    # Replace "/" with "-" in the season string
    season = season.replace("/", "-")
    # Create the directory path
    directory = os.path.join("output", "match", league, season, round, f'{home_name}_{away_name}')
    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Define the file path
    file_path_game = os.path.join(directory, f"heatmap_{pomSS}.png")
    plt.savefig(file_path_game, bbox_inches='tight')

    plt.close()
    return file_path_game

In [ ]:
heatmap(season=season)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/mplsoccer/_pitch_plot.py:170: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  return sns.kdeplot(x=x, y=y, ax=ax, clip=self.kde_clip, **kwargs)


'output/match/Eredivisie/2023-2024/Jornada 32/Feyenoord_PEC Zwolle/heatmap_Santiago Giménez.png'

### Calling Functions

In [ ]:
# Shots paths
file_path_player = None
if pom_position == 'Attacker' or pom_position == 'Midfielder':
    file_path_player = playerShots(season=season)
file_path_homeShots = homeShots(season=season)
file_path_awayShots = awayShots(season=season)

file_paths_shots = [file_path_player, file_path_homeShots, file_path_awayShots]
file_path_dashboard = dashboard(season=season)
file_path_heatmap = heatmap(season=season)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/mplsoccer/_pitch_plot.py:170: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  return sns.kdeplot(x=x, y=y, ax=ax, clip=self.kde_clip, **kwargs)
